In [1]:
import matplotlib.pyplot as plt
import numpy as np
import kinesis as kn
import gapipes as gp
import astropy.coordinates as coord
import astropy.units as u

In [2]:
def make_grid_cluster(ra, dec, d, v0, degsize=5.0):
    b0 = coord.ICRS(ra * u.deg, dec * u.deg, d * u.pc).cartesian.xyz.value

    ra_bins = np.linspace(ra - degsize, ra + degsize, 11)
    dec_bins = np.linspace(dec - degsize, dec + degsize, 11)
    ra_grid, dec_grid = np.meshgrid(ra_bins, dec_bins)
    ra_grid = ra_grid.ravel()
    dec_grid = dec_grid.ravel()
    memicrs = coord.ICRS(ra_grid * u.deg, dec_grid * u.deg, [d] * ra_grid.size * u.pc)

    cl = kn.Cluster(v0, 0.0, b0=b0).sample_at(memicrs)
    return cl

In [3]:
v0 = [5., 5., 7.07106781]
cl1 = make_grid_cluster(45, 45, 100, v0)

#10% error
cov = np.array([np.diag(X) for X in np.abs(cl1.members.truth[['parallax','pmra','pmdec']].values)*0.01])
cl1.members.observe(cov=cov, rv_error=[1]*cl1.N)

In [4]:
model = kn.get_model('allcombined')

INFO:kinesis.models:Reading model from disk


In [17]:
D = cl1.members.observed
irv = np.where(D['radial_velocity'].notnull().values)[0]
standata = dict(
    ra=D['ra'].values,
    dec=D['dec'].values,
    a=D[['parallax','pmra','pmdec']].values,
    C=D.g.make_cov(),
    N=len(D),
    Nrv=D['radial_velocity'].notnull().sum(),
    irv=irv,
    rv=D['radial_velocity'].values[irv],
    rv_error=D['radial_velocity_error'].values[irv],
    include_T=1,
    b0=cl1.b0
)
def init():
    return dict(d=1e3/D['parallax'],v0=[-2,3,0], sigv=[1,1,1], Omega=np.eye(3),T=np.zeros(shape=(1,3,3)),v0_bg=[0,0,0],f_mem=0.9,sigv_bg=50.0)

In [18]:
r = model.optimizing(data=standata, init=init)

In [19]:
r['v0']

array([-1.99927524e+00,  2.99906990e+00,  8.71244555e-05])

In [ ]:
r = model.sampling(data=standata, init=init)

In [10]:
r['sigv']

array([3.99493183e-05, 3.99325871e-05, 3.91479581e-04])